In [1]:
import os
par_dir = os.path.abspath(os.path.join(os.getcwd(),os.pardir))
os.chdir(par_dir)

In [2]:
import numpy as np
import pandas as pd
import joblib
x_data = np.load('data/metal.npy') # metal
x2_data = np.load('data/pretreatment.npy') # pretreatment
x3_data = np.load('data/reaction.npy') # reaction
c_data = np.load('data/support.npy') #support
### train_test_split을 활용해 데이터를 train,val,test로 해서 0.6,0.2,0.2로 나눠서 구분
# - random_state = 21로 설정
from sklearn.model_selection import train_test_split
x_train,x_test,x2_train,x2_test,x3_train,x3_test,c_train,c_test = train_test_split(x_data,x2_data,x3_data,c_data, random_state = 21,test_size = 0.4)
x_val,x_test,x2_val,x2_test,x3_val,x3_test,c_val,c_test = train_test_split(x_test,x2_test,x3_test,c_test,random_state = 21, test_size = 0.5)
# x_train,x_val,x_test = [np.log1p(x) for x in [x_train,x_val,x_test]]
# c_train,c_val,c_test = [np.log1p(x) for x in [c_train,c_val,c_test]]


In [3]:
from sklearn.preprocessing import StandardScaler,MaxAbsScaler
x_scaler = MaxAbsScaler()
x2_scaler = StandardScaler()
x3_scaler = StandardScaler()
c_scaler = MaxAbsScaler()
x_train = x_scaler.fit_transform(x_train)
x2_train = x2_scaler.fit_transform(x2_train)
x3_train = x3_scaler.fit_transform(x3_train)
c_train = c_scaler.fit_transform(c_train)

In [4]:
x_val,x_test = [x_scaler.transform(x) for x in [x_val,x_test]]
x2_val,x2_test = [x2_scaler.transform(x) for x in [x2_val,x2_test]]
x3_val,x3_test = [x3_scaler.transform(x) for x in [x3_val,x3_test]]
c_val,c_test = [c_scaler.transform(c) for c in [c_val,c_test]]
# import joblib 
joblib.dump(x_scaler,'./torch/a_27_x_scaler.pkl')
joblib.dump(x2_scaler,'./torch/s_27_x2_scaler.pkl')
joblib.dump(x3_scaler,'./torch/s_27_x3_scaler.pkl')
joblib.dump(c_scaler,'./torch/a_27_c_scaler.pkl')
import torch
x_train,x_val,x_test = [torch.tensor(x, dtype = torch.float32) for x in [x_train,x_val,x_test]]
x2_train,x2_val,x2_test = [torch.tensor(x, dtype = torch.float32) for x in [x2_train,x2_val,x2_test]]
x3_train,x3_val,x3_test = [torch.tensor(x, dtype = torch.float32) for x in [x3_train,x3_val,x3_test]]
c_train,c_val,c_test = [torch.tensor(c, dtype = torch.float32) for c in [c_train,c_val,c_test]]

In [5]:
from torch.utils.data import TensorDataset,DataLoader
train_data = [x_train,x2_train,x3_train, c_train]
val_data = [x_val,x2_val,x3_val,c_val]
test_data = [x_test, x2_test, x3_test, c_test]
train_data = TensorDataset(*train_data)
val_data = TensorDataset(*val_data)
test_data = TensorDataset(*test_data)
datas = [train_data,val_data,test_data]
train_loader,val_loader,test_loader = [DataLoader(x,batch_size = 64,shuffle=False) for x in datas]

In [6]:
import os
os.makedirs('torch',exist_ok = True)
torch.save(train_loader,"torch/train_loader_all.pt")
torch.save(val_loader,"torch/val_loader_all.pt")
torch.save(test_loader,"torch/test_loader_all.pt")